In [1]:
import matplotlib.pyplot as plt, time
import PyQt5

import sys, os, getopt, math, random
import numpy as np
from constants import *
%matplotlib qt
plt.ion()
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})


In [223]:
np.random.seed(1234)
swarm_size = 10  # default size of the swarm
manual_mode = False  # manually press enter key to proceed between simulations
def S14_closest_robot(robot_host, robot_neighbors):
    # "robot_host": the robot to measure distance from
    # "robot_neighbors": a list of robots to be compared with
    robot_closest = robot_neighbors[0]
    dist_closest = dist_table[robot_host,robot_closest]
    for i in robot_neighbors[1:]:
        dist_temp = dist_table[robot_host,i]
        if dist_temp < dist_closest:
            robot_closest = i
            dist_closest = dist_temp
    return robot_closest

def check_amoeba(poses_t, poses, num_robots, step, thresh = 1):
    """
    Function to check if a given amoeboid swarm is amoeboid after movement of the swarm robots

    Args:
        poses_t (np.array): Original positions of the robots
        poses (np.array): New positions of teh robors
        num_robots (int): Number of robots currently in the swarm
        step (float): step moving distance
        thresh (float, optional): threshold value for centroid and distance from centroid conditions. Defaults to 1.

    Returns:
        bool: True if the new swarm is amoeboid, False otherwise
    """    
    centroid = getcenteroid(poses[:num_robots])
    centroid_t = getcenteroid(poses_t[:num_robots])
    # print(centroid)
    arr_radii = np.array([get_dist(centroid, p) for p in poses[:num_robots]])
    arr_radii_t = np.array([get_dist(centroid, p) for p in poses_t[:num_robots]])
    
    delta_centroid = get_dist(centroid, centroid_t)
    delta_radius = np.mean(arr_radii) - np.mean(arr_radii_t)
    
    if delta_centroid/step < thresh and delta_radius/np.mean(arr_radii_t) < thresh:
        return True
    return False
    
# Function to create a unit vector from a given vector
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

# Function to get centroid
def getcenteroid(arr):
    length, dim = arr.shape
    return np.array([np.sum(arr[:, i])/length for i in range(dim)])

def get_dist(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def euclidean_distance(a, b):
    """Euclidean Distance

    Args:
        a (np.array): a
        b (np.array): b

    Returns:
        float: dist
    """
    return np.linalg.norm(a-b)


In [224]:
# general function to steer robot away from wall if out of boundary (following physics)
# use global variable "world_side_length"
def robot_boundary_check(robot_pos, robot_ori):
    world_side_length=30
    new_ori = robot_ori
    if robot_pos[0] >= world_side_length:  # outside of right boundary
        if math.cos(new_ori) > 0:
            new_ori = reset_radian(2*(math.pi/2) - new_ori)
            # further check if new angle is too much perpendicular
            if new_ori > 0:
                if (math.pi - new_ori) < perp_thres:
                    new_ori = new_ori - devia_angle
            else:
                if (new_ori + math.pi) < perp_thres:
                    new_ori = new_ori + devia_angle
    elif robot_pos[0] <= 0:  # outside of left boundary
        if math.cos(new_ori) < 0:
            new_ori = reset_radian(2*(math.pi/2) - new_ori)
            if new_ori > 0:
                if new_ori < perp_thres:
                    new_ori = new_ori + devia_angle
            else:
                if (-new_ori) < perp_thres:
                    new_ori = new_ori - devia_angle
    if robot_pos[1] >= world_side_length:  # outside of top boundary
        if math.sin(new_ori) > 0:
            new_ori = reset_radian(2*(0) - new_ori)
            if new_ori > -math.pi/2:
                if (new_ori + math.pi/2) < perp_thres:
                    new_ori = new_ori + devia_angle
            else:
                if (-math.pi/2 - new_ori) < perp_thres:
                    new_ori = new_ori - devia_angle
    elif robot_pos[1] <= 0:  # outside of bottom boundary
        if math.sin(new_ori) < 0:
            new_ori = reset_radian(2*(0) - new_ori)
            if new_ori > math.pi/2:
                if (new_ori - math.pi/2) < perp_thres:
                    new_ori = new_ori + devia_angle
            else:
                if (math.pi/2 - new_ori) < perp_thres:
                    new_ori = new_ori - devia_angle
    return new_ori

# # general function to steer robot away from wall if out of boundary (in random direction)
# # use global variable "world_side_length"
# def robot_boundary_check(robot_pos, robot_ori):
#     new_ori = robot_ori
#     if robot_pos[0] >= world_side_length:  # outside of right boundary
#         if math.cos(new_ori) > 0:
#             new_ori = reset_radian(math.pi/2 + np.random.uniform(0,math.pi))
#     elif robot_pos[0] <= 0:  # outside of left boundary
#         if math.cos(new_ori) < 0:
#             new_ori = reset_radian(-math.pi/2 + np.random.uniform(0,math.pi))
#     if robot_pos[1] >= world_side_length:  # outside of top boundary
#         if math.sin(new_ori) > 0:
#             new_ori = reset_radian(-math.pi + np.random.uniform(0,math.pi))
#     elif robot_pos[1] <= 0:  # outside of bottom boundary
#         if math.sin(new_ori) < 0:
#             new_ori = reset_radian(0 + np.random.uniform(0,math.pi))
#     return new_ori

# general function to reset radian angle to [-pi, pi)
def reset_radian(radian):
    while radian >= math.pi:
        radian = radian - 2*math.pi
    while radian < -math.pi:
        radian = radian + 2*math.pi
    return radian

In [225]:
# calculate world_side_coef from a desired screen size for 30 robots
def cal_world_side_coef():
    desired_screen_size = 500  # desired screen size for 30 robots
    desired_world_size = float(desired_screen_size) / pixels_per_length
    return desired_world_size / pow(30, 1/power_exponent)
world_side_coef = cal_world_side_coef()
# world_side_length = world_side_coef * pow(swarm_size, 1/power_exponent)
world_side_length = world_side_coef * pow(30, 1/power_exponent)
world_size = (world_side_length, world_side_length)  # square physical world
# screen size calculated from world size
screen_side_length = int(pixels_per_length * world_side_length)
screen_size = (screen_side_length, screen_side_length)  # square display world

In [580]:

def plot_swarm(robot_poses, size, prev=None):
    plt.clf()
    plt.rcParams["figure.figsize"] = [7.00, 7]
    plt.rcParams["figure.autolayout"] = True
    plt.xlim(0, 30)
    plt.ylim(0, 30)
    centroid = (np.sum(robot_poses[:, 0])/robot_poses.shape[0], np.sum(robot_poses[:, 1])/robot_poses.shape[0])
    plt.plot(centroid[0], centroid[1], marker="x")
    for robot in range(len(robot_poses[:size])):
        plt.plot(robot_poses[robot, 0], robot_poses[robot, 1], marker="o", markersize=4, markeredgecolor="red", markerfacecolor="green")
        
        # plot direction
        vector = np.array([math.cos(robot_oris[robot]), math.sin(robot_oris[robot])])
        # print(robot_poses[robot, 0], robot_poses[robot, 0]+vector[0])
        x_val = [robot_poses[robot, 0], robot_poses[robot, 0]+vector[0]]
        y_val = [robot_poses[robot, 1], robot_poses[robot, 1]+vector[1]]
        plt.plot(x_val, y_val)
        
    if prev is not None:
        for line in prev:
            x_val = [x[0] for x in line]
            y_val = [x[1] for x in line]
            plt.plot(x_val, y_val)
        
    # print(x_val)
# plot_swarm(robot_poses, current_swarm_size)

In [609]:
def get_swarm_vector(robot_oris):
    vector = np.array([0.0,0.0])
    for ori in robot_oris:
        vec = np.array([math.cos(ori), math.sin(ori)])
        vector += vec
    return vector

def get_radial_distance(robot_poses, current_swarm_size):
    centroid = np.array([np.sum(robot_poses[:, 0]), np.sum(robot_poses[:, 1])])/current_swarm_size
    radial_dist = 0
    for robot in range(current_swarm_size):
        radial_dist+= euclidean_distance(centroid, robot_poses[robot])
        
    return radial_dist

In [582]:
# screen.fill(color_white)

# robot properties
# robot_poses = np.random.rand(swarm_size, 2) * world_side_length  # initialize the robot poses
robot_poses = np.array([[1.0,1.0] for i in range(swarm_size)])  # initialize the robot poses
dist_table = np.zeros((swarm_size, swarm_size))  # distances between robots
conn_table = np.zeros((swarm_size, swarm_size))  # connections between robots
    # 0 for disconnected, 1 for connected
conn_lists = [[] for i in range(swarm_size)]  # lists of robots connected
# function for all simulations, update the distances and connections between the robots
def dist_conn_update():
    global dist_table
    global conn_table
    global conn_lists
    conn_lists = [[] for i in range(swarm_size)]  # empty the lists
    for i in range(swarm_size):
        for j in range(i+1, swarm_size):
            dist_temp = np.linalg.norm(robot_poses[i] - robot_poses[j])
            dist_table[i,j] = dist_temp
            dist_table[j,i] = dist_temp
            if dist_temp > comm_range:
                conn_table[i,j] = 0
                conn_table[j,i] = 0
            else:
                conn_table[i,j] = 1
                conn_table[j,i] = 1
                conn_lists[i].append(j)
                conn_lists[j].append(i)
dist_conn_update()  # update the distances and connections
disp_poses = []  # display positions
# function for all simulations, update the display positions
def disp_poses_update():
    global disp_poses
    poses_temp = robot_poses / world_side_length
    poses_temp[:,1] = 1.0 - poses_temp[:,1]
    poses_temp = poses_temp * screen_side_length
    disp_poses = poses_temp.astype(int)  # convert to int and assign to disp_poses
    # disp_poses = np.array([[0,0] for i in range(swarm_size)])  # 
disp_poses_update()
# deciding the seed robots, used in simulations with moving robots
seed_percentage = 0.1  # the percentage of seed robots in the swarm
seed_quantity = min(max(int(swarm_size*seed_percentage), 1), swarm_size)
    # no smaller than 1, and no larger than swarm_size
robot_seeds = [False for i in range(swarm_size)]  # whether a robot is a seed robot
    # only seed robot can initialize the forming a new group
seed_list_temp = np.arange(swarm_size)
np.random.shuffle(seed_list_temp)
for i in seed_list_temp[:seed_quantity]:
    robot_seeds[i] = True
    
current_swarm_size = 1  # the current swarm size
sim_haulted = False
# set up the simulation window

disp_poses_update()

# robot_oris = np.random.rand(swarm_size) * 2 * math.pi - math.pi  # in range of [-pi, pi)

robot_oris = np.random.rand(swarm_size) * math.pi/2   # in range of [0, pi/2)
# use step moving distance in each update, instead of calculating from robot velocity
# so to make it independent of simulation frequency control
step_moving_dist = 0.05  # should be smaller than destination distance error
destination_error = 0.1
mov_vec_ratio = 0.5  # ratio used when calculating mov vector
# spring constants in SMA
linear_const = 1.0
bend_const = 0.8
disp_coef = 0.5
# for avoiding space too small on loop
space_good_thres = desired_space * 0.85

past = []
plot_swarm(robot_poses, current_swarm_size)

In [614]:
step = 5.0
# init = 100
delta_c = 1.5
delta_r = 1.5
step_normal = 3.0
swarm_ori = np.random.random()*math.pi

for iter in range(10):

    centroid = np.array([np.sum(robot_poses[:, 0]), np.sum(robot_poses[:, 1])])/current_swarm_size
    # print(centroid)
    robot_poses_t = robot_poses.copy()  # stores the old copy of robot positions
    if current_swarm_size == 0: pass #continue
    elif current_swarm_size < 3:
        old_centroid = np.array([1,1])
        # random_vector = unit_vector((np.random.rand(2)-0.5)*2)
        for j in range(current_swarm_size):
            # point = np.random.randint(0, 100, size=2)
            # print(point)
            # robot_oris[j] = math.atan2(point[0], point[1])
            # if current_swarm_size < swarm_size:
            #     robot_oris[j] = np.absolute(robot_oris[j])
        
            robot_oris[j] = robot_boundary_check(robot_poses_t[j], robot_oris[j])
            robot_poses[j] = robot_poses_t[j] + (step/5 * np.array([math.cos(robot_oris[j]), math.sin(robot_oris[j])]))
            past.append((robot_poses[j], robot_poses_t[j]))
        old_centroid = centroid
        changed_distance = euclidean_distance(old_centroid, centroid)
        old_radial = get_radial_distance(robot_poses, current_swarm_size)
        
    else:
        # Add robot to swarm
        if current_swarm_size < swarm_size:
            # print("Adding one robot")
            # point = np.random.randint(0, 100, size=2)
            # print(point)
            j = current_swarm_size
            # robot_oris[j] = math.atan2(point[0], point[1])
            robot_oris[j] = robot_boundary_check(robot_poses_t[j], robot_oris[j])
            robot_poses[j] = robot_poses_t[j] + (step * np.array([math.cos(robot_oris[j]), math.sin(robot_oris[j])]))
        # Pseudopod Grouping
        
        
        robot_ids = list(range(current_swarm_size))
        for div in range(current_swarm_size//3):
            init = np.random.choice(robot_ids)
            # print("Robot chosen is: ", init)
            robot_ids.remove(init)
            closest = S14_closest_robot(init, robot_ids)
            robot_ids.remove(closest)
            # print("closest robot id is: ", closest)
            
            # Choose another robot that is closest to the two
            closest1 = S14_closest_robot(init, robot_ids)
            closest2 = S14_closest_robot(closest, robot_ids)
            if closest1 == closest2:
                triad = [init, closest, closest1]
            else:
                if dist_table[init, closest1] < dist_table[closest, closest2]:
                    triad = [init, closest, closest1]
                    robot_ids.remove(closest1)
                else:
                    triad = [init, closest, closest2]
                    robot_ids.remove(closest2)
            # print("triad is: ", triad) # Direction of Last robot in this list is updated, other two are pseudopods
            dist = [euclidean_distance(robot_poses[triad[i]], centroid) for i in range(len(triad))]
            
            # dist.sort(reverse=True)
            vector1 = np.array([math.cos(robot_oris[triad[0]]), math.sin(robot_oris[triad[0]])])
            vector2 = np.array([math.cos(robot_oris[triad[1]]), math.sin(robot_oris[triad[1]])])
            
            current_vector = np.array([math.cos(robot_oris[triad[2]]), math.sin(robot_oris[triad[2]])])
            random_ori = np.random.random()* math.pi / 2
            random_vector = np.array([math.cos(random_ori), math.sin(random_ori)])
            centroid_vector = np.array(centroid) - np.array(robot_poses[triad[2]])
            avg_vector = get_swarm_vector(robot_oris)
            
            vector = 4*np.random.random()*(vector1 + vector2) + 2 * np.random.random()* current_vector + np.random.random() * centroid_vector + np.random.random() * avg_vector + np.random.random()*random_vector
            
            
            robot_oris[triad[2]] = math.atan2(vector[1], vector[0])
            # print(vector)
            # robot_oris[triad[2]] = math.atan2(new[1], new[0])
            
            step_dist = 0
            coord_old = robot_poses[triad[2]]
            
            sum_coord = sum(robot_poses[:current_swarm_size])
            
            robot_oris[triad[2]] = robot_boundary_check(robot_poses_t[triad[2]], robot_oris[triad[2]])
            
            for step_candidate in np.linspace(step/2, step, 10):
                new_pos = robot_poses_t[triad[2]] + (step_candidate * np.array([math.cos(robot_oris[triad[2]]), math.sin(robot_oris[triad[2]])]))
                new_centroid = np.array(np.sum(new_pos), np.sum(new_pos))/current_swarm_size
                # print(euclidean_distance(old_centroid, centroid))
                if euclidean_distance(new_centroid, centroid)/changed_distance < delta_c:
                    break
                
            changed_distance = euclidean_distance(new_centroid, centroid) - euclidean_distance(old_centroid, centroid)
            robot_poses[triad[2]] = new_pos
            centroid = np.array([np.sum(robot_poses[:, 0]), np.sum(robot_poses[:, 1])])/current_swarm_size
            past.append((robot_poses[triad[2]], robot_poses_t[triad[2]]))
            
            # robot_oris[triad[2]] = robot_boundary_check(robot_poses_t[triad[2]], robot_oris[triad[2]])
            # robot_poses[triad[2]] = robot_poses_t[triad[2]] + (step * np.array([math.cos(robot_oris[triad[2]]), math.sin(robot_oris[triad[2]])]))
            
        curr_radial_dist = get_radial_distance(robot_poses, current_swarm_size)
        # print(robot_ids)
        robot_ids.sort(key=lambda x: get_dist(centroid, robot_poses[x]), reverse=True)
        for robot in robot_ids:
            current_vector = np.array([math.cos(robot_oris[robot]), math.sin(robot_oris[robot])])
            centroid_vector = np.array(centroid) - np.array(robot_poses[robot])
            final_vector = np.random.random()* current_vector+np.random.random()*centroid_vector
            
            ori = math.atan2(final_vector[1], final_vector[0])
            ori = robot_boundary_check(robot_poses_t[robot], ori)
            
            for step_candidate in np.linspace(step, step/2, 10):
                new_pos = robot_poses_t[robot] + (step_candidate * np.array([math.cos(ori), math.sin(ori)]))
                new_radial = curr_radial_dist - euclidean_distance(centroid, robot_poses_t[robot])+ euclidean_distance(centroid, new_pos)
                
                if (new_radial)/old_radial < delta_r:
                    robot_poses[robot] = new_pos
                    past.append((robot_poses[robot], robot_poses_t[robot]))

            centroid = np.array([np.sum(robot_poses[:, 0]), np.sum(robot_poses[:, 1])])/current_swarm_size
            
        
        old_radial = new_radial
    old_centroid = centroid
    prev = robot_poses_t[:(current_swarm_size-1)] if current_swarm_size < swarm_size else robot_poses_t
    print("previous robot positions: ", prev)         
    print("robot positions: ", robot_poses[:current_swarm_size]) 
    
    if current_swarm_size < swarm_size:
        current_swarm_size+=1 
    dist_conn_update()
    disp_poses_update()
    plt.clf()
    plot_swarm(robot_poses, current_swarm_size, past)
    # time.sleep(1)
    # disp_poses=robot_poses

previous robot positions:  [[28.221 29.123]
 [21.956 31.104]
 [26.760 30.368]
 [24.984 28.914]
 [28.341 28.526]
 [28.420 28.564]
 [26.757 29.854]
 [32.146 29.511]
 [23.088 32.103]
 [27.781 29.094]]
robot positions:  [[30.638 28.483]
 [24.358 30.411]
 [26.760 30.368]
 [27.484 28.919]
 [28.341 28.526]
 [28.420 28.564]
 [26.757 29.854]
 [32.146 29.511]
 [25.356 31.052]
 [25.383 29.803]]
previous robot positions:  [[30.638 28.483]
 [24.358 30.411]
 [26.760 30.368]
 [27.484 28.919]
 [28.341 28.526]
 [28.420 28.564]
 [26.757 29.854]
 [32.146 29.511]
 [25.356 31.052]
 [25.383 29.803]]
robot positions:  [[28.287 29.333]
 [24.358 30.411]
 [25.425 28.254]
 [29.965 28.609]
 [28.341 28.526]
 [28.420 28.564]
 [26.757 29.854]
 [29.718 28.915]
 [25.319 28.552]
 [25.383 29.803]]
previous robot positions:  [[28.287 29.333]
 [24.358 30.411]
 [25.425 28.254]
 [29.965 28.609]
 [28.341 28.526]
 [28.420 28.564]
 [26.757 29.854]
 [29.718 28.915]
 [25.319 28.552]
 [25.383 29.803]]
robot positions:  [[25.788 2

In [140]:
unit_vector(np.random.randint(-100, 100, size=2))

array([-0.35340057, -0.93547209])

In [328]:
np.array([get_dist(centroid, p) for p in robot_poses_t[:current_swarm_size]])

array([0.391, 0.566, 1.430, 0.487, 0.893, 1.000, 1.854])

In [91]:
np.array([np.sum(robot_poses[:, 0]), np.sum(robot_poses[:, 1])])/current_swarm_size

array([5.670, 6.373])

In [337]:
np.linspace(2.5, 5, 10)

array([2.500, 2.778, 3.056, 3.333, 3.611, 3.889, 4.167, 4.444, 4.722,
       5.000])

In [40]:
np.array(centroid)/current_swarm_size

array([5.825, 5.277])

In [327]:
np.random.random()

0.5220880797206794

In [344]:
7//3


2

In [540]:
x = [(1,2), (3, 4)]
[a[0] for a in x]

[1, 3]